### Selfie2Anime

In [2]:
# Library Import
import glob
import random
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import math
import itertools
import datetime
import time
from torchvision.utils import save_image, make_grid
from torchvision import datasets
from torch.autograd import Variable

In [4]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print('using: ',device)

using:  cuda


In [5]:
# 흑백이미지를 RGB 이미지로 바꾸는 함수
def to_rgb(image):
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image

In [6]:
# 사용자 정의 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned
        # train 모드일 때는 trainA, trainB에 있는 디렉토리에서 이미지를 불러옵니다. 
        if mode=="train":
            # glob 함수로 trainA 디렉토리의 이미지의 목록을 불러옵니다. 
            self.files_A = sorted(glob.glob(os.path.join(root, "trainA") + "/*.*"))
            self.files_B = sorted(glob.glob(os.path.join(root, "trainB") + "/*.*"))
        else:
            self.files_A = sorted(glob.glob(os.path.join(root, "testA") + "/*.*"))
            self.files_B = sorted(glob.glob(os.path.join(root, "testB") + "/*.*"))
    
    def __getitem__(self, index):
        # index값으로 이미지의 목록 중 이미지 하나를 불러옵니다. 
        image_A = Image.open(self.files_A[index % len(self.files_A)])
        # unaligned 변수로 학습할 Pair를 랜덤으로 고릅니다.
        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.file_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # Convert Grayscale images to rgb
        if image_A.mode != "RGB":
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB":
            image_B = to_rgb(image_B)
        # 불러온 PIL 이미지를 우리가 인자로 받은 transform함수를 적용해서 torch tensor자료형으로 변환
        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return {"A": item_A, "B": item_B}
    
    # 정의한 데이터셋으로 Loader를 이용해 배치 사이즈만큼 이미지를 불러올 수 있다.
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

#### Generator 구현

In [7]:
# 가중치 초기화 함수
# torch에서 제공하는 layer의 종류에 따라 가중치 초기화를 다르게 해서 종류에 맞게 가중치를 초기화한다.
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [8]:
# Residual block 구현
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            # Reflectionpadding은 점대칭 방식으로 가장 가까운 픽셀로부터 값을 복사해온다. 더욱 자연스러운 이미지생성을 위해서
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            # InstanceNormalization은 데이터개별 정규화이며, 데이터 범위를 비슷하게 만들어주는 것이다.
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        # Residual Block이 늘어날수록 더 많은 계층정보를 바탕으로 그럴듯한 이미지가 생성됩니다.
        return x + self.block(x)

In [9]:
# Generator 구현
# 제너레이터의 흐름( 입력이미지 -> 다운샘플링 -> 여러개의 Residual Block통과 -> 업샘플링 )

class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()
        channels = input_shape[0]

        # 초기 Convolution block 선언
        out_features = 64
        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features

        # 다운샘플링을 2번 진행한다. stride=2이므로 이미지가 반으로 줄어든다.
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
        
        # num_residual_block만큼 residual block
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)]

        # nn업샘플링을 2번 진행하여 다시 이미지의 크기를 2배씩 늘려준다.
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
        # Output layer 출력 레이어를 선언 출력이미지는 입력이미지와 크기가 동일하고 활성화함수는 탄젠트를 사용한다.
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [10]:
# Discriminator 구현
# discriminator는 입력받은 이미지가 실제 이미지인지 생성이미지인지 분류하는 역할을 한다.
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        channels, height, width = input_shape
        # discriminator의 출력크기를 정의한다.
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)
        def discriminator_block(in_filters, out_filters, normalize=True):
            # discriminator block은 stride=2로 점점 다운샘플링하며 출력 이미지의 크기를 줄인다.
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2,
                                                                padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        # 이미지 크기가 256*256일때 discriminatorblock을 4번 통과하면 16*16이 된다. (1번 통과할 때마다 크기가 반으로 줄어 )
        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )
    def forward(self, img):
        return self.model(img)

In [11]:
# 모델학습
# HyperParameter지정

# 학습 및 테스트 데이터가 들어가 있는 폴더를 의미
dataset_name = 'self2anime'
# 이밈지의 채널 수를 의미하며 흑백은1, RGB는 3이다.
channels = 3
img_height = 256
img_width = 256
# Generator에서 Residual Block의 개수를 의미
n_residual_blocks=9
# Learning Rate
lr = 0.0002
# b1, b2는 Adam Optimizer에 대한 하이퍼파라미터
b1 = 0.5
b2 = 0.999
n_epochs=200
init_epoch=0
decay_epoch=100
lambda_cyc = 10.0
# lambda_id의 람다값이 클수록 본래의 색감을 유지하려는 성질이 있다.
lambda_id = 5.0
n_cpu = 8
batch_size = 1
sample_interval = 100
checkpoint_interval = 5

In [12]:
# Create sample and checkpoint directories
# 샘플이미지와 모델 가중치를 저장할 폴더 생성
os.makedirs("images/%s" % dataset_name, exist_ok=True)
os.makedirs("saved_models/%s" % dataset_name, exist_ok=True)

In [13]:
# 손실함수 정의
# Losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

In [14]:
# 모델 객체 선언하기
input_shape = (channels, img_height, img_width)

# A에서 B로 변환하는 G_AB
# B에서 A로 변환하는 G_BA
# 생성한 스타일이 진짜인지판별하는 D_A, D_B
# Initialize generator and discriminator
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_BA = GeneratorResNet(input_shape, n_residual_blocks)
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)

In [15]:
print(G_AB)
# print(G_BA)
# print(D_A)
# print(D_B)

GeneratorResNet(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResidualBlock(
      (block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        